In [197]:

# general data and data viz imports
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from copy import copy
# adjusting display restrictions 
pd.options.display.max_columns = 100
pd.options.display.max_rows = 3000
# setting preference to remove scientific notation 
pd.options.display.float_format = '{:,.2f}'.format

%matplotlib inline

[ U.S. Drought Monitor Data Dictionary](https://droughtmonitor.unl.edu/DmData/DataDownload/WebServiceInfo.aspx#comp)


Important Notes:

pulled the percentage of area to minimize impact of large magnitude for in models



In [198]:
drought = pd.read_csv('../data/drought_index_by_state.csv')
drought.head()

,MapDate,StateAbbreviation,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20220405,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-04-05,2022-04-11,2
1,20220329,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-29,2022-04-04,2
2,20220322,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-22,2022-03-28,2
3,20220315,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-15,2022-03-21,2
4,20220308,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-08,2022-03-14,2


In [199]:
drought.columns=drought.columns.str.lower()
drought.columns[:2]

Index(['mapdate', 'stateabbreviation'], dtype='object')

In [200]:
# extracting month and year from 
sliced_month=[]
sliced_year=[]
for  obs in drought['mapdate']:
    obs=str(obs)
    month= int(obs[4:6])
    year=int(obs[:4])
    sliced_month.append(month)
    sliced_year.append(year)

drought['month'] = sliced_month
drought['year'] = sliced_year
drought.head()

,mapdate,stateabbreviation,none,d0,d1,d2,d3,d4,validstart,validend,statisticformatid,month,year
0,20220405,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-04-05,2022-04-11,2,4,2022
1,20220329,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-29,2022-04-04,2,3,2022
2,20220322,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-22,2022-03-28,2,3,2022
3,20220315,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-15,2022-03-21,2,3,2022
4,20220308,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-08,2022-03-14,2,3,2022


#### Computing DSCI using categorical USDM statistics
[Reference](https://droughtmonitor.unl.edu/About/AbouttheData/DSCI.aspx)

In [201]:
drought['DSCI']=1*(drought['d0'])+2*drought['d1']+3*drought['d2']+4*drought['d3'] + 5*drought['d3']
drought.head()

,mapdate,stateabbreviation,none,d0,d1,d2,d3,d4,validstart,validend,statisticformatid,month,year,DSCI
0,20220405,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-04-05,2022-04-11,2,4,2022,0.00
1,20220329,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-29,2022-04-04,2,3,2022,0.00
2,20220322,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-22,2022-03-28,2,3,2022,0.00
3,20220315,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-15,2022-03-21,2,3,2022,0.00
4,20220308,AK,100.00,0.00,0.00,0.00,0.00,0.00,2022-03-08,2022-03-14,2,3,2022,0.00


In [202]:
# no duplicates
drought[drought.duplicated()]

,mapdate,stateabbreviation,none,d0,d1,d2,d3,d4,validstart,validend,statisticformatid,month,year,DSCI


In [203]:
print('this is daily: ',drought.shape)

this is daily:  (16276, 14)


In [204]:
grouped_drought = drought.groupby(['year','month','stateabbreviation'],
                                  as_index=False).mean()

In [205]:
print('this is monthly:',grouped_drought.shape)

this is monthly: (3796, 12)


In [206]:
grouped_drought.head()

,year,month,stateabbreviation,mapdate,none,d0,d1,d2,d3,d4,statisticformatid,DSCI
0,2016,4,AK,"20,160,419.00",77.38,22.62,0.00,0.00,0.00,0.00,2.00,22.62
1,2016,4,AL,"20,160,419.00",97.04,2.96,0.00,0.00,0.00,0.00,2.00,2.96
2,2016,4,AR,"20,160,419.00",78.71,21.29,0.00,0.00,0.00,0.00,2.00,21.29
3,2016,4,AZ,"20,160,419.00",0.47,32.78,65.58,1.17,0.00,0.00,2.00,167.45
4,2016,4,CA,"20,160,419.00",4.01,5.74,15.88,23.18,26.60,24.59,2.00,346.40


In [207]:
grouped_drought.tail()

,year,month,stateabbreviation,mapdate,none,d0,d1,d2,d3,d4,statisticformatid,DSCI
3791,2022,4,VT,"20,220,405.00",63.91,36.09,0.00,0.00,0.00,0.00,2.00,36.09
3792,2022,4,WA,"20,220,405.00",44.07,5.92,19.57,20.88,9.56,0.00,2.00,193.74
3793,2022,4,WI,"20,220,405.00",49.61,33.50,16.90,0.00,0.00,0.00,2.00,67.30
3794,2022,4,WV,"20,220,405.00",33.01,65.70,1.29,0.00,0.00,0.00,2.00,68.28
3795,2022,4,WY,"20,220,405.00",0.00,2.85,37.23,39.69,20.23,0.00,2.00,378.45


In [208]:
# finally filter out unnecessary columns
grouped_drought= grouped_drought[['year','month','stateabbreviation','d0','d1','d2','d3','d4','DSCI']]
grouped_drought.rename(columns={'stateabbreviation': 'state','DSCI':'DSCI_avg'}, inplace=True)
grouped_drought.head()

,year,month,state,d0,d1,d2,d3,d4,DSCI_avg
0,2016,4,AK,22.62,0.00,0.00,0.00,0.00,22.62
1,2016,4,AL,2.96,0.00,0.00,0.00,0.00,2.96
2,2016,4,AR,21.29,0.00,0.00,0.00,0.00,21.29
3,2016,4,AZ,32.78,65.58,1.17,0.00,0.00,167.45
4,2016,4,CA,5.74,15.88,23.18,26.60,24.59,346.40


## Below is FAQ Directly from 'About the Data' page on US Drought Monitor Site
Q: How can I summarize the U.S. Drought Monitor over time, with one number?

A: Best practices for using DSCI over time are still evolving and depend on the intended use. The most obvious choices are to average or sum it.

Averaging DSCI: The advantage of averaging the DSCI is that the same 0-500 scale applies, so users would not have to learn a new scale, and less qualifying detail is needed for context. For example, a state climatologist might find it useful to compare the average DSCI for their state from month to month as part of a regular report.

Summing DSCI: Summing the DSCI may be more useful for comparing different droughts. If for example two areas experienced droughts of comparable duration, but one had a higher total DSCI than the other, we could conclude that the one with the higher DSCI experienced a more intense or more widespread drought. Summing the DSCI could also be useful for comparing two droughts from the same area.

In [209]:
SUM_grouped_drought = drought.groupby(['year','month','stateabbreviation'],
                                  as_index=False).sum()
# appending the more acceptable numerical representation of DSCI (sum)
grouped_drought['DSCI_summed'] = SUM_grouped_drought['DSCI']
grouped_drought.head()

,year,month,state,d0,d1,d2,d3,d4,DSCI_avg,DSCI_summed
0,2016,4,AK,22.62,0.00,0.00,0.00,0.00,22.62,67.86
1,2016,4,AL,2.96,0.00,0.00,0.00,0.00,2.96,8.89
2,2016,4,AR,21.29,0.00,0.00,0.00,0.00,21.29,63.86
3,2016,4,AZ,32.78,65.58,1.17,0.00,0.00,167.45,502.34
4,2016,4,CA,5.74,15.88,23.18,26.60,24.59,346.40,"1,039.21"


In [210]:
grouped_drought.dtypes

year             int64
month            int64
state           object
d0             float64
d1             float64
d2             float64
d3             float64
d4             float64
DSCI_avg       float64
DSCI_summed    float64
dtype: object

In [211]:
# exporting without index 
grouped_drought.to_csv('../data/clean_dIndex_by_state_mth.csv',index_label=False)

# For Readme

[imagelink](https://droughtmonitor.unl.edu/About/AbouttheData/DroughtClassification.aspx)

### Drought Index Classification values 
These classifcations (D0 to D4) were provided from the US Drought Monitor, a partnership between US Department of Agriculture, NOAA, and National Drought and Mitigation Center at the University of Nebraska-Lincoln. 

# ![](Viz/readme_viz/drought_classfication_image.png)

The US Drought monitor descibes a high level look at drought conditions across the US and thus is not specific to local conditions, however this is a similar scale as to the wildfire data sources so could be indicative of fire severity. 

The drought conditions of interest to aid in predicting wildfire risk were specifically drought intensity. Where D1 is the least intensive and D4 is the most intensive extreme. D0 signifies the area is not in a drought, however there are abnormal dry conditions that are at risk of meeting minimum drought conditions. The other potential case are those areas that could be recovering from drought but have yet to reach normal conditions, which is a case that we would need to distinguish between during EDA.

The intensity categories were created based on the following sources:
- original 5 key indicators (WHAT are those) including other objective indicators
- local condition reports and impact reports 450+ expert observers nationwide
- drought impacts (unsure what this is....)




Originally for drought data scrubbing:
- Could not find anything that discussed what 